# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Dataset & Modeling</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/3_training_dataset_and_modeling.ipynb)

<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups
2. Define Transformation functions
4. Create Feature Views
5. Create Training Dataset with training, validation and test splits

![part2](../../images/02_training-dataset.png) 

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U hopsworks --quiet

!pip install xgboost --q

In [ ]:
import os
import datetime
import time
import json
import pickle
import joblib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather',
    version = 1
)

--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_except(['unix_time']), on=['city_name', 'date'])

In [ ]:
# here you can check out the merged dataframe

# query_df = query.read()

In [ ]:
# query_df.city_name.value_counts()

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

You can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='air_quality_fv',
    version=1,
    query=query
)

For now `Feature View` is saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view()`.

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset you use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- You can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

In [ ]:
td_version, td_job = feature_view.create_training_data(
    description = 'Air Quality dataset',
    data_format = 'csv',
    write_options = {'wait_for_job': True}
)

### <span style="color:#ff5f27;">🪝 Training Dataset Retrieval</span>

In [ ]:
X, _ = feature_view.get_training_data(
    training_dataset_version=td_version
)

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
label_encoder = LabelEncoder()

# Fit the encoder to the data
label_encoder.fit(X[['city_name']])

# Transform the data
encoded = label_encoder.transform(X[['city_name']])

In [ ]:
# Convert the output to a dense array and concatenate with the original data
X = pd.concat([X, pd.DataFrame(encoded)], axis=1)

X = X.drop(columns=['date', 'city_name', 'unix_time'])
X = X.rename(columns={0: "city_name_encoded"})

In [ ]:
y = X.pop('pm2_5')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

### <span style='color:#ff5f27'> ⚖️ Model Validation

In [ ]:
start_of_cell = time.time()

xgb_regressor = XGBRegressor()
xgb_regressor.fit(X_train, y_train)

y_pred = xgb_regressor.predict(X_test)

# calculate MSE using sklearn
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# calculate RMSE using sklearn
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

# calculate R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

end_of_cell = time.time()
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

In [ ]:
df_ = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred
})

In [ ]:
residplot = sns.residplot(data=df_, x="y_true", y="y_pred", color='orange')
plt.title('Model Residuals')
plt.xlabel('Obsevation #')
plt.ylabel('Error')

plt.show()
fig = residplot.get_figure()
fig.show()

In [ ]:
from xgboost import plot_importance


plot_importance(xgb_regressor, max_num_features=25)

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
# The 'nyc_taxi_fares_model' directory will be saved to the model registry
model_dir="air_quality_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(label_encoder, model_dir + '/label_encoder.pkl')
joblib.dump(xgb_regressor, model_dir + '/xgboost_regressor.pkl')

fig.savefig(model_dir + "/residplot.png") 

In [ ]:
aq_model = mr.python.create_model(
    name="air_quality_xgboost_model", 
    metrics={
        "RMSE": rmse,
        "MSE": mse,
        "R squared": r2
    },
    model_schema=model_schema,
    input_example=X_test.sample().values, 
    description="Air Quality (PM2.5) predictor.")

aq_model.save(model_dir)

---

## <span style="color:#ff5f27;">🪝 Retrieving model from Model Registry</span>

In [ ]:
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1
)

saved_model_dir = retrieved_model.download()

In [ ]:
retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

In [ ]:
retrieved_xgboost_model

## <span style="color:#ff5f27;">✨ Load Batch Data of last days</span>

First, you will need to fetch the training dataset that you created in the previous notebook.

In [ ]:
today = datetime.date.today()
date_threshold = today - datetime.timedelta(days=30)
str(date_threshold)

In [ ]:
start_of_cell = time.time()


feature_view.init_batch_scoring(training_dataset_version=td_version)

batch_data = feature_view.get_batch_data(start_time=date_threshold)


end_of_cell = time.time()
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
# Transform the data
encoded = retrieved_encoder.transform(batch_data['city_name'])

# Convert the output to a dense array and concatenate with the original data
X_batch = pd.concat([batch_data, pd.DataFrame(encoded)], axis=1)

X_batch = X_batch.drop(columns=['date', 'city_name', 'unix_time'])
X_batch = X_batch.rename(columns={0: 'city_name_encoded'})

y_batch = X_batch.pop('pm2_5')

In [ ]:
predictions = retrieved_xgboost_model.predict(X_batch)
predictions[:5]

---

## <span style="color:#ff5f27;">👾 Now try out the Streamlit App!</span>

In [ ]:
# install dependcies
!pip3 install geopy streamlit streamlit-folium folium  --q

In [ ]:
!python3 -m streamlit run streamlit_app.py

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps </b> </span>

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai